In [1]:
import tensorflow as tf
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras import layers
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import img_to_array, array_to_img
from mtcnn import MTCNN
import cv2

2024-12-19 05:11:18.923172: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-12-19 05:11:18.924570: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-12-19 05:11:19.473721: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-12-19 05:11:19.702997: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1734559879.818691    6324 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1734559879.82

In [2]:
print(tf.__version__)

2.18.0


In [3]:
detector = MTCNN()

2024-12-19 05:11:28.028480: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


In [4]:
class L2NormalizeLayer(layers.Layer):
    def call(self, inputs):
        return tf.math.l2_normalize(inputs, axis=1)

class AbsoluteDifferenceLayer(layers.Layer):
    def call(self, inputs):
        return tf.abs(inputs[0] - inputs[1])

In [5]:
model = load_model('siamese_baru_lagi.keras', custom_objects={'L2NormalizeLayer': L2NormalizeLayer, 'AbsoluteDifferenceLayer': AbsoluteDifferenceLayer})

In [6]:
model.input_shape

[(None, 160, 160, 3), (None, 160, 160, 3)]

In [7]:
def detect_and_crop_face(image_path, output_size=(160, 160)):
    """
    Deteksi wajah, crop, dan resize gambar menjadi ukuran output_size.
    """
    # Baca gambar menggunakan OpenCV
    img = cv2.imread(image_path)
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Konversi BGR ke RGB
    
    # Deteksi wajah
    faces = detector.detect_faces(img_rgb)
    if faces:
        # Ambil bounding box wajah pertama
        x, y, width, height = faces[0]['box']
        x, y = max(0, x), max(0, y)  # Pastikan koordinat tidak negatif
        
        # Crop wajah
        cropped_face = img_rgb[y:y+height, x:x+width]
        
        # Resize menjadi output_size
        resized_face = cv2.resize(cropped_face, output_size)

        normalized_face = np.array(resized_face) / 255.0  # Normalisasi

        return normalized_face  # Gambar wajah yang sudah diproses
    else:
        return None  # Tidak ada wajah yang terdeteksi

In [36]:
image_1 = detect_and_crop_face('dataset_copy/train/Achmad Raihan/IMG_20241117_161935.jpg')
image_2 = detect_and_crop_face('jokowi_1.jpeg')

In [37]:
image_1 = np.expand_dims(image_1, axis=0)  # Tambahkan dimensi batch
image_2 = np.expand_dims(image_2, axis=0)  # Tambahkan dimensi batch


In [38]:
image_1.shape

(1, 160, 160, 3)

In [39]:
image_1 = tf.convert_to_tensor(image_1, dtype=tf.float32)
image_2 = tf.convert_to_tensor(image_2, dtype=tf.float32)

In [40]:
image_1.shape

TensorShape([1, 160, 160, 3])

In [41]:
predictions = model.predict([image_1, image_2])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 266ms/step


In [42]:
print(predictions)

[[0.4094489]]
